In [1]:
%load_ext autoreload
%autoreload 2
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import processing_netcdf as pcdf
import geopandas as gpd
from geopandas import GeoDataFrame
import shapely.geometry 
import numpy as np
from shapely import geometry as gmty
from geofeather import to_geofeather, from_geofeather

In [10]:
import glob
import os

In [3]:
#path for 11 models
path = "/home/mlopez/EXEC/Tg_annual_11_models/"

In [11]:
files = list(glob.glob(os.path.join(path,'*.*')))

In [12]:
files

['/home/mlopez/EXEC/Tg_annual_11_models/ACCESS1-3_rcp45_tg_mean_annual.nc',
 '/home/mlopez/EXEC/Tg_annual_11_models/BNU-ESM_rcp45_tg_mean_annual.nc',
 '/home/mlopez/EXEC/Tg_annual_11_models/CanESM2_rcp45_tg_mean_annual.nc',
 '/home/mlopez/EXEC/Tg_annual_11_models/CMCC-CMS_rcp45_tg_mean_annual.nc',
 '/home/mlopez/EXEC/Tg_annual_11_models/GFDL-ESM2M_rcp45_tg_mean_annual.nc',
 '/home/mlopez/EXEC/Tg_annual_11_models/HadGEM2-CC_rcp45_tg_mean_annual.nc',
 '/home/mlopez/EXEC/Tg_annual_11_models/INM-CM4_rcp45_tg_mean_annual.nc',
 '/home/mlopez/EXEC/Tg_annual_11_models/IPSL-CM5A-LR_rcp45_tg_mean_annual.nc',
 '/home/mlopez/EXEC/Tg_annual_11_models/MPI-ESM-LR_rcp45_tg_mean_annual.nc',
 '/home/mlopez/EXEC/Tg_annual_11_models/NorESM1-M_rcp45_tg_mean_annual.nc',
 '/home/mlopez/EXEC/Tg_annual_11_models/NRCAN_obs_tg_mean_annual.nc']

# Create DF of all models

In [16]:
dflist = []

In [17]:
%time for file in files:
    df = pcdf.load_as_df(file)
    dflist.append(df)

CPU times: user 6 µs, sys: 3 µs, total: 9 µs
Wall time: 17.9 µs


/exec/mlopez/Code/GitHub/processing_netcdf.py:24: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  ds['time'] = index.to_datetimeindex()
/exec/mlopez/Code/GitHub/processing_netcdf.py:24: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  ds['time'] = index.to_datetimeindex()
/exec/mlopez/Code/GitHub/processing_netcdf.py:24: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, '360_day', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between

In [18]:
dflist

[                                    tg_mean
 lat       lon        time                  
 66.623306 -89.045212 1950-01-01  258.828888
                      1951-01-01  258.898285
                      1952-01-01  258.709839
                      1953-01-01  258.461304
                      1954-01-01  259.764862
 ...                                     ...
 40.041039 -74.045807 2096-01-01  287.267639
                      2097-01-01  287.343201
                      2098-01-01  289.033142
                      2099-01-01  287.839813
                      2100-01-01  287.281799
 
 [11507106 rows x 1 columns],                                     tg_mean
 lat       lon        time                  
 66.623306 -89.045212 1950-01-01  257.955170
                      1951-01-01  256.791321
                      1952-01-01  259.168579
                      1953-01-01  258.776642
                      1954-01-01  257.527466
 ...                                     ...
 40.041039 -74.045807 20

In [21]:
%time df11 = pd.concat(dflist)

CPU times: user 3min 14s, sys: 5min 15s, total: 8min 29s
Wall time: 59.6 s


In [22]:
df11

tg_mean
lat       lon        time                  
66.623306 -89.045212 1950-01-01  258.828888
                     1951-01-01  258.898285
                     1952-01-01  258.709839
                     1953-01-01  258.461304
                     1954-01-01  259.764862
...                                     ...
41.708332 -82.208336 2009-01-01  282.584686
                     2010-01-01  283.870300
                     2011-01-01  284.144257
                     2012-01-01  285.570129
                     2013-01-01  283.616577

[131535892 rows x 1 columns]

In [23]:
dftgm = df11.reset_index()

In [26]:
dftgm

,lat,lon,time,tg_mean
0,66.623306,-89.045212,1950-01-01,258.828888
1,66.623306,-89.045212,1951-01-01,258.898285
2,66.623306,-89.045212,1952-01-01,258.709839
3,66.623306,-89.045212,1953-01-01,258.461304
4,66.623306,-89.045212,1954-01-01,259.764862
...,...,...,...,...
131535887,41.708332,-82.208336,2009-01-01,282.584686
131535888,41.708332,-82.208336,2010-01-01,283.870300
131535889,41.708332,-82.208336,2011-01-01,284.144257
131535890,41.708332,-82.208336,2012-01-01,285.570129


# Get the mean for 11 models

In [28]:
%time dftgm2 = dftgm.groupby(["lat", "lon", "time"]).mean()

CPU times: user 47.5 s, sys: 1min 17s, total: 2min 4s
Wall time: 1min 39s


In [29]:
dftgm2

tg_mean
lat       lon        time                  
40.041039 -89.045212 1950-01-01  284.431244
                     1951-01-01  284.329865
                     1952-01-01  284.034576
                     1953-01-01  284.014282
                     1954-01-01  284.159058
...                                     ...
83.125000 -69.625000 2009-01-01  256.029419
                     2010-01-01  257.224335
                     2011-01-01  256.532013
                     2012-01-01  256.204498
                     2013-01-01  254.789719

[27971938 rows x 1 columns]

# Transform temp to Celsius

In [35]:
dfC = dftgm2.copy()
dfC["tg_mean"] = dfC["tg_mean"] -273.15
    

# Define 3 periods of time: 

In [36]:
year_groups = {y:0 for y in range(1980,2011)}
year_groups.update({y:1 for y in range(2041,2071)})
year_groups.update({y:2 for y in range(2071,2101)})
#Get the mean of every period
dfC2 = dfC.reset_index()
dfp = dfC2.groupby([dfC2.time.dt.year.map(year_groups), "lat","lon"]).mean()

In [38]:
dfp.reset_index()

,time,lat,lon,tg_mean
0,0.0,40.041039,-89.045212,11.578949
1,0.0,40.041039,-88.961884,11.559995
2,0.0,40.041039,-88.878548,11.550130
3,0.0,40.041039,-88.795219,11.530299
4,0.0,40.041039,-88.711891,11.547827
...,...,...,...,...
485876,2.0,66.623306,-61.629639,-6.880292
485877,2.0,66.623306,-61.546310,-7.195539
485878,2.0,66.623306,-61.462978,-6.991460
485879,2.0,66.623306,-61.379650,-6.924743


# Historic Period

In [41]:
df_h = dfp.query("time==0.0")
df_h.reset_index()

,time,lat,lon,tg_mean
0,0.0,40.041039,-89.045212,11.578949
1,0.0,40.041039,-88.961884,11.559995
2,0.0,40.041039,-88.878548,11.550130
3,0.0,40.041039,-88.795219,11.530299
4,0.0,40.041039,-88.711891,11.547827
...,...,...,...,...
333464,0.0,83.125000,-69.958336,-19.014936
333465,0.0,83.125000,-69.875000,-18.887545
333466,0.0,83.125000,-69.791672,-18.915213
333467,0.0,83.125000,-69.708336,-18.884167


## Import dataframe with polygons

In [43]:
dfpolyshape = from_geofeather('Grid-TerritoiresGuides.feather')

/home/mlopez/EXEC/anaconda3/envs/python37-2/lib/python3.8/site-packages/geofeather/core.py:85: UserWarning: Grid-TerritoiresGuides.feather.crs coordinate reference system file is missing. No crs will be set for this GeoDataFrame.
  warnings.warn(


In [44]:
dfpolyshape

,lat,lon,TER_GUIDE,geometry
0,52.957191,-67.712730,6opqr,"POLYGON ((-67.67107 52.93424, -67.67107 52.915..."
1,52.957191,-67.629402,6opqr,"POLYGON ((-67.67107 52.91553, -67.67107 52.934..."
2,52.957191,-67.546066,6opqr,"POLYGON ((-67.56730 52.91553, -67.57974 52.915..."
3,52.873859,-67.796059,6opqr,"POLYGON ((-67.75439 52.89329, -67.75439 52.832..."
4,52.873859,-67.712730,6opqr,"POLYGON ((-67.75439 52.83219, -67.75439 52.893..."
...,...,...,...,...
33193,45.041668,-72.208336,2c,"POLYGON ((-72.25000 45.00450, -72.25000 45.083..."
33194,45.041668,-72.125000,2c,"POLYGON ((-72.16666 45.00508, -72.16666 45.083..."
33195,45.041668,-72.041672,2c,"POLYGON ((-72.08334 45.00564, -72.08334 45.083..."
33196,45.041668,-71.958336,2c,"POLYGON ((-72.00000 45.00667, -72.00000 45.083..."


## Merge data with mask 

In [51]:
dftg11TG = pd.merge(df_h, dfpolyshape, on=["lat","lon"])

In [56]:
dftg11TG.dropna()

,lat,lon,tg_mean,TER_GUIDE,geometry
0,44.957512,-74.629120,7.226765,1a,"POLYGON ((-74.62590 44.99918, -74.58746 44.999..."
1,44.957512,-74.545792,7.201068,1a,"POLYGON ((-74.58746 44.99876, -74.58746 44.999..."
2,44.957512,-74.462463,7.176799,1a,"POLYGON ((-74.50413 44.99784, -74.50413 44.999..."
3,44.957512,-74.379128,7.162602,1a,"POLYGON ((-74.42079 44.99530, -74.42079 44.999..."
4,44.957512,-74.295799,7.125215,1a,"POLYGON ((-74.33746 44.99215, -74.33746 44.999..."
...,...,...,...,...,...
33127,52.957191,-67.629402,-3.079251,6opqr,"POLYGON ((-67.67107 52.91553, -67.67107 52.934..."
33128,52.957191,-67.546066,-3.065557,6opqr,"POLYGON ((-67.56730 52.91553, -67.57974 52.915..."
33129,52.958332,-67.708336,-3.045003,6opqr,"POLYGON ((-67.66667 52.93419, -67.66667 52.916..."
33130,52.958332,-67.625000,-2.945693,6opqr,"POLYGON ((-67.66666 52.91667, -67.66666 52.934..."


In [57]:
dftg11TG.TER_GUIDE.value_counts()

6cdefg    4720
6opqr     3218
6ab       2836
5bcd      2468
4bc       2105
6hi       2054
6j        1899
6mn       1848
3ab       1304
5a        1102
6kl       1019
5ef        800
4f         783
2b         776
3d         732
3c         711
4de        711
5g         697
5hi        683
1a         678
4gh        605
5jk        489
2c         331
2a         285
4a         278
Name: TER_GUIDE, dtype: int64

In [59]:
TG4a = dftg11TG[dftg11TG.TER_GUIDE == "4a"]
TG4a

,lat,lon,tg_mean,TER_GUIDE,geometry
5240,47.124088,-79.462257,3.679755,4a,"POLYGON ((-79.44652 47.16575, -79.42059 47.165..."
5242,47.124088,-79.378929,3.539073,4a,"POLYGON ((-79.42059 47.15410, -79.42059 47.165..."
5244,47.124088,-79.295601,3.344405,4a,"POLYGON ((-79.33237 47.16575, -79.29047 47.165..."
5388,47.125000,-79.458336,3.742499,4a,"POLYGON ((-79.44617 47.16667, -79.41667 47.166..."
5390,47.125000,-79.375000,3.614611,4a,"MULTIPOLYGON (((-79.33473 47.16667, -79.33334 ..."
...,...,...,...,...,...
9620,48.207378,-79.045609,2.125691,4a,"POLYGON ((-79.08727 48.16571, -79.08727 48.178..."
9622,48.207378,-78.962280,2.145050,4a,"POLYGON ((-79.00394 48.16571, -79.00394 48.178..."
9824,48.208332,-79.125000,2.223469,4a,"POLYGON ((-79.08334 48.17644, -79.08334 48.166..."
9826,48.208332,-79.041672,2.270755,4a,"POLYGON ((-79.08334 48.16667, -79.08334 48.176..."


In [60]:
TG4a.to_csv("/home/mlopez/EXEC/Tg_csv/TG4a11.csv")

# ---- Create GeoJson file ----

In [47]:
geometry = dftg11TG["geometry"]

In [48]:
crs = {'init': "epsg:4326"}
gdf = GeoDataFrame(dftg11TG, crs=crs, geometry=geometry)

In [49]:
gdf.to_file("TG11tg.json", driver="GeoJSON")